The method introduced in [CV strategy](https://www.kaggle.com/its7171/cv-strategy) is very helpful, but it seems that the users are slightly biased.

In [ ]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%%time
train = pd.read_pickle('../input/riiid-cross-validation-files/cv1_train.pickle')
valid = pd.read_pickle('../input/riiid-cross-validation-files/cv1_valid.pickle')
train = pd.concat([train, valid])
del valid
gc.collect()

In [ ]:
interval = 10000000
user_count = []
for i in range(10):
    start = i * interval
    user_count.append(train[start:start+interval].user_id.nunique())

In [ ]:
plt.bar(list(range(10)), user_count)
plt.show()

The bar plot above shows how many users exist in each of the 10 million. It seems that there are many users in the first and last 10 million. Something is wrong.

In [ ]:
last10m = train[-interval:]
interval = 1000000
user_count = []
for i in range(10):
    start = i * interval
    user_count.append(last10m[start:start+interval].user_id.nunique())

In [ ]:
plt.bar(list(range(10)), user_count)
plt.show()

The bar plot above shows how many users exist in the last 10 million, separated by 1 million. It seems that there are many users in the last million. Why is this happening?

In [ ]:
max_timestamp_u = train[['user_id','timestamp']].groupby(['user_id']).agg(['max']).reset_index()
max_timestamp_u.columns = ['user_id', 'max_time_stamp']
max_timestamp_u.set_index('user_id', inplace=True)
max_timestamp_u.head()

In [ ]:
last100m = train[-100000000:]
interval = 2500000
mean_max_timestamp = []
for i in range(40):
    start = i * interval
    user_list = last100m[start:start+interval].user_id.unique()
    mean_max_timestamp.append(max_timestamp_u.reindex(user_list).mean())
mean_max_timestamp = pd.concat(mean_max_timestamp)

In [ ]:
plt.bar(list(range(40)), mean_max_timestamp)
plt.show()

The bar plot above is the average of the user's maximum timestamps, separated by 2.5 million of the last 100 million. It seems that the sooner and the later users have shorter time stamps.

Many users tackle many problems at the beginning, but the number of problems tends to decrease after that. Maybe that's the cause.

You should be aware that there are differences between users of training data and validation data.